In [1]:
import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')
from hparams import create_hparams
from model import Tacotron2
import torch
from loss_function import Tacotron2Loss
from train import prepare_dataloaders
import waveglow.glow
from waveglow.denoiser import Denoiser
import IPython.display as ipd
import sys
from text import cmudict, text_to_sequence
from data_utils import TextMelLoader, TextMelCollate
import waveglow
import librosa
from layers import TacotronSTFT
from train import warm_start_model
from torch.utils.data import DataLoader
from train import save_checkpoint, load_checkpoint

sys.path.append('waveglow/')
def load_mel(path):
    audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
    audio = torch.from_numpy(audio)
    if sampling_rate != hparams.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = melspec.cuda()
    return melspec


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model

arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow).cuda().eval()
#from glow import infer

hparams = create_hparams(None)
hparams.n_speakers = 1
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/lj_val.txt'

# hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt'
# hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'
hparams.batch_size = 16
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
learning_rate = 1e-3#

stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)

criterion = Tacotron2Loss()
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
iteration = 0
epoch_offset = 0
is_overflow = False

#i, batch = next(enumerate(train_loader))


torch.manual_seed(hparams.seed)
model = load_model(hparams)

#train
model = warm_start_model(
    hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16


model.eval()
with torch.no_grad():
    val_sampler = None
    batch_size = 4
    val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                            shuffle=False, batch_size=batch_size,
                            pin_memory=False, 
                            collate_fn=collate_fn)

    val_loss = 0.0
    i, batch = next(enumerate(val_loader))
    x, y = model.parse_batch(batch)
    y_pred = model(x)
    loss = criterion(y_pred, y)

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
here


In [2]:
# hparams.n_speakers = 3
# torch.manual_seed(hparams.seed)
# model = load_model(hparams)
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
# #eval
# checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uber3_5iter'
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
# optimizer.eps = 1e-16
# model, optimizer, _learning_rate, iteration = load_checkpoint(
#                 checkpoint_path, model, optimizer)


In [3]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.validation_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
text = "its like this and like that and like this and uh"
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )
text = to_arpabet(text)
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)
#print(audio_path, text)


#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

Warning! Reached max decoder steps


In [4]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [6]:
torch.cuda.empty_cache()
from numba import cuda 
import torch
#cuda.select_device(0)
#cuda.close()
cuda.select_device(0)
cuda.close()
torch.cuda.empty_cache()

In [11]:
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 1], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [12]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [13]:
# import torch
# torch.cuda.empty_cache()

In [6]:
#eval
checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uber3_5iter'
optimizer.eps = 1e-16
model, optimizer, _learning_rate, iteration = load_checkpoint(checkpoint_path, model, optimizer)
#

Loading checkpoint '/mnt/disks/uberduck-experiments-v0/uber3_5iter'
Loaded checkpoint '/mnt/disks/uberduck-experiments-v0/uber3_5iter' from iteration 5


In [8]:
# import sys
# !{sys.executable} -m pip install g2p_en

In [7]:
    model.eval()
    with torch.no_grad():
        val_sampler = None
        batch_size = 4
        val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                                shuffle=False, batch_size=batch_size,
                                pin_memory=False, 
                                collate_fn=collate_fn)

        val_loss = 0.0
        i, batch = next(enumerate(val_loader))
        x, y = model.parse_batch(batch)
        y_pred = model(x)
        loss = criterion(y_pred, y)
        
# embedded_gst = model.gst(x[2], x[4])
# embedded_inputs = model.embedding(text_encoded).transpose(1, 2)
# embedded_text = model.encoder.inference(embedded_inputs)
# embedded_gst = embedded_gst.repeat(1, embedded_text.size(1), 1)

/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here


In [ ]:
#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([2], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [ ]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([1], device='cuda:0').to(torch.int64)
xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [ ]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [14]:
#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([2], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros(x[6][[0]].shape, device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [15]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [16]:
speaker_ids = torch.tensor([1], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros(x[6][[0]].shape, device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [17]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [18]:
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros(x[6][[0]].shape, device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [19]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [ ]:
x[6].shape

In [ ]:
# AUTOGENERATED! DO NOT EDIT! File to edit: nbs/models.mellotron.ipynb (unless otherwise specified).

__all__ = ['DEFAULTS', 'Postnet', 'Prenet', 'Encoder', 'Decoder', 'Tacotron2']

# Cell
from math import sqrt

import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from .base import TTSModel
from .common import Attention, Conv1d, LinearNorm, GST
from ..text.symbols import symbols
from ..vendor.tfcompat.hparam import HParams
from ..utils import to_gpu, get_mask_from_lengths


DEFAULTS = HParams(
    n_symbols=len(symbols),
    symbols_embedding_dim=512,
    mask_padding=True,
    fp16_run=False,
    n_mel_channels=80,
    # encoder parameters
    encoder_kernel_size=5,
    encoder_n_convolutions=3,
    encoder_embedding_dim=512,
    # decoder parameters
    n_frames_per_step=1,  # currently only 1 is supported
    decoder_rnn_dim=1024,
    prenet_dim=256,
    prenet_f0_n_layers=1,
    prenet_f0_dim=1,
    prenet_f0_kernel_size=1,
    prenet_rms_dim=0,
    prenet_fms_kernel_size=1,
    max_decoder_steps=1000,
    gate_threshold=0.5,
    p_attention_dropout=0.1,
    p_decoder_dropout=0.1,
    p_teacher_forcing=1.0,
    # attention parameters
    attention_rnn_dim=1024,
    attention_dim=128,
    # location layer parameters
    attention_location_n_filters=32,
    attention_location_kernel_size=31,
    # mel post-processing network parameters
    postnet_embedding_dim=512,
    postnet_kernel_size=5,
    postnet_n_convolutions=5,
    # speaker_embedding
    n_speakers=123,  # original nvidia libritts training
    speaker_embedding_dim=128,
    # reference encoder
    with_gst=True,
    ref_enc_filters=[32, 32, 64, 64, 128, 128],
    ref_enc_size=[3, 3],
    ref_enc_strides=[2, 2],
    ref_enc_pad=[1, 1],
    ref_enc_gru_size=128,
    # style token layer
    token_embedding_size=256,
    token_num=10,
    num_heads=8,
    include_f0=False,
)

# Cell


class Postnet(nn.Module):
    """Postnet
    - Five 1-d convolution with 512 channels and kernel size 5
    """

    def __init__(self, hparams):
        super(Postnet, self).__init__()
        self.dropout_rate = 0.5
        self.convolutions = nn.ModuleList()

        self.convolutions.append(
            nn.Sequential(
                Conv1d(
                    hparams.n_mel_channels,
                    hparams.postnet_embedding_dim,
                    kernel_size=hparams.postnet_kernel_size,
                    stride=1,
                    padding=int((hparams.postnet_kernel_size - 1) / 2),
                    dilation=1,
                    w_init_gain="tanh",
                ),
                nn.BatchNorm1d(hparams.postnet_embedding_dim),
            )
        )

        for i in range(1, hparams.postnet_n_convolutions - 1):
            self.convolutions.append(
                nn.Sequential(
                    Conv1d(
                        hparams.postnet_embedding_dim,
                        hparams.postnet_embedding_dim,
                        kernel_size=hparams.postnet_kernel_size,
                        stride=1,
                        padding=int((hparams.postnet_kernel_size - 1) / 2),
                        dilation=1,
                        w_init_gain="tanh",
                    ),
                    nn.BatchNorm1d(hparams.postnet_embedding_dim),
                )
            )

        self.convolutions.append(
            nn.Sequential(
                Conv1d(
                    hparams.postnet_embedding_dim,
                    hparams.n_mel_channels,
                    kernel_size=hparams.postnet_kernel_size,
                    stride=1,
                    padding=int((hparams.postnet_kernel_size - 1) / 2),
                    dilation=1,
                    w_init_gain="linear",
                ),
                nn.BatchNorm1d(hparams.n_mel_channels),
            )
        )

    def forward(self, x):
        for i in range(len(self.convolutions) - 1):
            x = F.dropout(torch.tanh(self.convolutions[i](x)), self.dropout_rate, self.training)
        x = F.dropout(self.convolutions[-1](x), self.dropout_rate, self.training)

        return x

# Cell

class Prenet(nn.Module):
    def __init__(self, in_dim, sizes):
        super(Prenet, self).__init__()
        in_sizes = [in_dim] + sizes[:-1]
        self.layers = nn.ModuleList(
            [
                LinearNorm(in_size, out_size, bias=False)
                for (in_size, out_size) in zip(in_sizes, sizes)
            ]
        )
        self.dropout_rate = 0.5

    def forward(self, x):
        for linear in self.layers:
            x = F.dropout(F.relu(linear(x)), p=self.dropout_rate, training=True)
        return x

# Cell
class Encoder(nn.Module):
    """Encoder module:
    - Three 1-d convolution banks
    - Bidirectional LSTM
    """

    def __init__(self, hparams):
        super(Encoder, self).__init__()

        convolutions = []
        for _ in range(hparams.encoder_n_convolutions):
            conv_layer = nn.Sequential(
                Conv1d(
                    hparams.encoder_embedding_dim,
                    hparams.encoder_embedding_dim,
                    kernel_size=hparams.encoder_kernel_size,
                    stride=1,
                    padding=int((hparams.encoder_kernel_size - 1) / 2),
                    dilation=1,
                    w_init_gain="relu",
                ),
                nn.BatchNorm1d(hparams.encoder_embedding_dim),
            )
            convolutions.append(conv_layer)
        self.convolutions = nn.ModuleList(convolutions)
        self.dropout_rate = 0.5

        self.lstm = nn.LSTM(
            hparams.encoder_embedding_dim,
            int(hparams.encoder_embedding_dim / 2),
            1,
            batch_first=True,
            bidirectional=True,
        )

    def forward(self, x, input_lengths):
        if x.size()[0] > 1:
            print("here")
            x_embedded = []
            for b_ind in range(x.size()[0]):  # TODO: Speed up
                curr_x = x[b_ind : b_ind + 1, :, : input_lengths[b_ind]].clone()
                for conv in self.convolutions:
                    curr_x = F.dropout(F.relu(conv(curr_x)), self.dropout_rate, self.training)
                x_embedded.append(curr_x[0].transpose(0, 1))
            x = torch.nn.utils.rnn.pad_sequence(x_embedded, batch_first=True)
        else:
            for conv in self.convolutions:
                x = F.dropout(F.relu(conv(x)), self.dropout_rate, self.training)
            x = x.transpose(1, 2)

        # pytorch tensor are not reversible, hence the conversion
        input_lengths = input_lengths.cpu().numpy()
        x = nn.utils.rnn.pack_padded_sequence(x, input_lengths, batch_first=True)

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)

        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return outputs

    def inference(self, x):
        for conv in self.convolutions:
            x = F.dropout(F.relu(conv(x)), self.dropout_rate, self.training)

        x = x.transpose(1, 2)

        self.lstm.flatten_parameters()
        outputs, _ = self.lstm(x)

        return outputs

# Cell

class Decoder(nn.Module):
    def __init__(self, hparams):
        super(Decoder, self).__init__()
        self.n_mel_channels = hparams.n_mel_channels
        self.n_frames_per_step = hparams.n_frames_per_step
        self.encoder_embedding_dim = (
            hparams.encoder_embedding_dim
            + hparams.token_embedding_size
            + hparams.speaker_embedding_dim
        )
        self.attention_rnn_dim = hparams.attention_rnn_dim
        self.decoder_rnn_dim = hparams.decoder_rnn_dim
        self.prenet_dim = hparams.prenet_dim
        self.max_decoder_steps = hparams.max_decoder_steps
        self.gate_threshold = hparams.gate_threshold
        self.p_attention_dropout = hparams.p_attention_dropout
        self.p_decoder_dropout = hparams.p_decoder_dropout
        self.p_teacher_forcing = hparams.p_teacher_forcing
        self.include_f0 = hparams.include_f0

        if self.include_f0:
            prenet_f0_dim = hparams.prenet_f0_dim
            self.prenet_f0 = Conv1d(
                1,
                prenet_f0_dim,
                kernel_size=hparams.prenet_f0_kernel_size,
                padding=max(0, int(hparams.prenet_f0_kernel_size / 2)),
                bias=False,
                stride=1,
                dilation=1,
            )
        else:
            prenet_f0_dim = 0

        self.prenet = Prenet(
            hparams.n_mel_channels * hparams.n_frames_per_step,
            [hparams.prenet_dim, hparams.prenet_dim],
        )

        self.attention_rnn = nn.LSTMCell(
            hparams.prenet_dim + prenet_f0_dim + self.encoder_embedding_dim,
            hparams.attention_rnn_dim,
        )

        self.attention_layer = Attention(
            hparams.attention_rnn_dim,
            self.encoder_embedding_dim,
            hparams.attention_dim,
            hparams.attention_location_n_filters,
            hparams.attention_location_kernel_size,
        )

        self.decoder_rnn = nn.LSTMCell(
            hparams.attention_rnn_dim + self.encoder_embedding_dim,
            hparams.decoder_rnn_dim,
            1,
        )

        self.linear_projection = LinearNorm(
            hparams.decoder_rnn_dim + self.encoder_embedding_dim,
            hparams.n_mel_channels * hparams.n_frames_per_step,
        )

        self.gate_layer = LinearNorm(
            hparams.decoder_rnn_dim + self.encoder_embedding_dim,
            1,
            bias=True,
            w_init_gain="sigmoid",
        )

    def get_go_frame(self, memory):
        """Gets all zeros frames to use as first decoder input
        PARAMS
        ------
        memory: decoder outputs
        RETURNS
        -------
        decoder_input: all zeros frames
        """
        B = memory.size(0)
        decoder_input = Variable(
            memory.data.new(B, self.n_mel_channels * self.n_frames_per_step).zero_()
        )
        return decoder_input

    def get_end_f0(self, f0s):
        B = f0s.size(0)
        dummy = Variable(f0s.data.new(B, 1, f0s.size(1)).zero_())
        return dummy

    def initialize_decoder_states(self, memory, mask):
        """Initializes attention rnn states, decoder rnn states, attention
        weights, attention cumulative weights, attention context, stores memory
        and stores processed memory
        PARAMS
        ------
        memory: Encoder outputs
        mask: Mask for padded data if training, expects None for inference
        """
        B = memory.size(0)
        MAX_TIME = memory.size(1)

        self.attention_hidden = Variable(
            memory.data.new(B, self.attention_rnn_dim).zero_()
        )
        self.attention_cell = Variable(
            memory.data.new(B, self.attention_rnn_dim).zero_()
        )

        self.decoder_hidden = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())
        self.decoder_cell = Variable(memory.data.new(B, self.decoder_rnn_dim).zero_())

        self.attention_weights = Variable(memory.data.new(B, MAX_TIME).zero_())
        self.attention_weights_cum = Variable(memory.data.new(B, MAX_TIME).zero_())
        self.attention_context = Variable(
            memory.data.new(B, self.encoder_embedding_dim).zero_()
        )

        self.memory = memory
        self.processed_memory = self.attention_layer.memory_layer(memory)
        self.mask = mask

    def parse_decoder_inputs(self, decoder_inputs):
        """Prepares decoder inputs, i.e. mel outputs
        PARAMS
        ------
        decoder_inputs: inputs used for teacher-forced training, i.e. mel-specs
        RETURNS
        -------
        inputs: processed decoder inputs
        """
        # (B, n_mel_channels, T_out) -> (B, T_out, n_mel_channels)
        decoder_inputs = decoder_inputs.transpose(1, 2)
        decoder_inputs = decoder_inputs.view(
            decoder_inputs.size(0),
            int(decoder_inputs.size(1) / self.n_frames_per_step),
            -1,
        )
        # (B, T_out, n_mel_channels) -> (T_out, B, n_mel_channels)
        decoder_inputs = decoder_inputs.transpose(0, 1)
        return decoder_inputs

    def parse_decoder_outputs(self, mel_outputs, gate_outputs, alignments):
        """Prepares decoder outputs for output
        PARAMS
        ------
        mel_outputs:
        gate_outputs: gate output energies
        alignments:
        RETURNS
        -------
        mel_outputs:
        gate_outpust: gate output energies
        alignments:
        """
        # (T_out, B) -> (B, T_out)
        alignments = torch.stack(alignments).transpose(0, 1)
        # (T_out, B) -> (B, T_out)
        gate_outputs = torch.stack(gate_outputs)
        if len(gate_outputs.size()) > 1:
            gate_outputs = gate_outputs.transpose(0, 1)
        else:
            gate_outputs = gate_outputs[None]
        gate_outputs = gate_outputs.contiguous()
        # (T_out, B, n_mel_channels) -> (B, T_out, n_mel_channels)
        mel_outputs = torch.stack(mel_outputs).transpose(0, 1).contiguous()
        # decouple frames per step
        mel_outputs = mel_outputs.view(mel_outputs.size(0), -1, self.n_mel_channels)
        # (B, T_out, n_mel_channels) -> (B, n_mel_channels, T_out)
        mel_outputs = mel_outputs.transpose(1, 2)

        return mel_outputs, gate_outputs, alignments

    def decode(self, decoder_input, attention_weights=None):
        """Decoder step using stored states, attention and memory
        PARAMS
        ------
        decoder_input: previous mel output
        RETURNS
        -------
        mel_output:
        gate_output: gate output energies
        attention_weights:
        """
        cell_input = torch.cat((decoder_input, self.attention_context), -1)
        self.attention_hidden, self.attention_cell = self.attention_rnn(
            cell_input, (self.attention_hidden, self.attention_cell)
        )
        self.attention_hidden = F.dropout(
            self.attention_hidden, self.p_attention_dropout, self.training
        )
        self.attention_cell = F.dropout(
            self.attention_cell, self.p_attention_dropout, self.training
        )

        attention_weights_cat = torch.cat(
            (
                self.attention_weights.unsqueeze(1),
                self.attention_weights_cum.unsqueeze(1),
            ),
            dim=1,
        )
        self.attention_context, self.attention_weights = self.attention_layer(
            self.attention_hidden,
            self.memory,
            self.processed_memory,
            attention_weights_cat,
            self.mask,
            attention_weights,
        )

        self.attention_weights_cum += self.attention_weights
        decoder_input = torch.cat((self.attention_hidden, self.attention_context), -1)
        self.decoder_hidden, self.decoder_cell = self.decoder_rnn(
            decoder_input, (self.decoder_hidden, self.decoder_cell)
        )
        self.decoder_hidden = F.dropout(
            self.decoder_hidden, self.p_decoder_dropout, self.training
        )
        self.decoder_cell = F.dropout(
            self.decoder_cell, self.p_decoder_dropout, self.training
        )

        decoder_hidden_attention_context = torch.cat(
            (self.decoder_hidden, self.attention_context), dim=1
        )

        decoder_output = self.linear_projection(decoder_hidden_attention_context)

        gate_prediction = self.gate_layer(decoder_hidden_attention_context)
        return decoder_output, gate_prediction, self.attention_weights

    def forward(self, memory, decoder_inputs, memory_lengths, f0s):
        """Decoder forward pass for training
        PARAMS
        ------
        memory: Encoder outputs
        decoder_inputs: Decoder inputs for teacher forcing. i.e. mel-specs
        memory_lengths: Encoder output lengths for attention masking.
        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """

        decoder_input = self.get_go_frame(memory).unsqueeze(0)
        decoder_inputs = self.parse_decoder_inputs(decoder_inputs)
        decoder_inputs = torch.cat((decoder_input, decoder_inputs), dim=0)
        decoder_inputs = self.prenet(decoder_inputs)

        # audio features
        if self.include_f0:
            f0_dummy = self.get_end_f0(f0s)
            f0s = torch.cat((f0s, f0_dummy), dim=2)
            f0s = F.relu(self.prenet_f0(f0s))
            f0s = f0s.permute(2, 0, 1)

        self.initialize_decoder_states(
            memory, mask=~get_mask_from_lengths(memory_lengths)
        )

        mel_outputs, gate_outputs, alignments = [], [], []
        while len(mel_outputs) < decoder_inputs.size(0) - 1:
            if (
                len(mel_outputs) == 0
                or np.random.uniform(0.0, 1.0) <= self.p_teacher_forcing
            ):
                if self.include_f0:
                    to_concat = (decoder_inputs[len(mel_outputs)], f0s[len(mel_outputs)])
                else:
                    to_concat = (decoder_inputs[len(mel_outputs)],)
                decoder_input = torch.cat(to_concat, dim=1)
            else:
                if self.include_f0:
                    to_concat = (self.prenet(mel_outputs[-1]), f0s[len(mel_outputs)])
                else:
                    to_concat = (self.prenet(mel_outputs[-1]),)
                decoder_input = torch.cat(to_concat, dim=1)
            mel_output, gate_output, attention_weights = self.decode(decoder_input)
            mel_outputs += [mel_output.squeeze(1)]
            gate_outputs += [gate_output.squeeze()]
            alignments += [attention_weights]

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

    def inference(self, memory, f0s):
        """Decoder inference
        PARAMS
        ------
        memory: Encoder outputs
        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        decoder_input = self.get_go_frame(memory)

        self.initialize_decoder_states(memory, mask=None)
        f0_dummy = self.get_end_f0(f0s)
        f0s = torch.cat((f0s, f0_dummy), dim=2)
        f0s = F.relu(self.prenet_f0(f0s))
        f0s = f0s.permute(2, 0, 1)

        mel_outputs, gate_outputs, alignments = [], [], []
        while True:
            if len(mel_outputs) < len(f0s):
                f0 = f0s[len(mel_outputs)]
            else:
                f0 = f0s[-1] * 0

            decoder_input = torch.cat((self.prenet(decoder_input), f0), dim=1)
            mel_output, gate_output, alignment = self.decode(decoder_input)

            mel_outputs += [mel_output.squeeze(1)]
            gate_outputs += [gate_output]
            alignments += [alignment]

            if torch.sigmoid(gate_output.data) > self.gate_threshold:
                break
            elif len(mel_outputs) == self.max_decoder_steps:
                print("Warning! Reached max decoder steps")
                break

            decoder_input = mel_output

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

    def inference_noattention(self, memory, f0s, attention_map):
        """Decoder inference
        PARAMS
        ------
        memory: Encoder outputs
        RETURNS
        -------
        mel_outputs: mel outputs from the decoder
        gate_outputs: gate outputs from the decoder
        alignments: sequence of attention weights from the decoder
        """
        decoder_input = self.get_go_frame(memory)

        self.initialize_decoder_states(memory, mask=None)
        f0_dummy = self.get_end_f0(f0s)
        f0s = torch.cat((f0s, f0_dummy), dim=2)
        f0s = F.relu(self.prenet_f0(f0s))
        f0s = f0s.permute(2, 0, 1)

        mel_outputs, gate_outputs, alignments = [], [], []
        for i in range(len(attention_map)):
            f0 = f0s[i]
            attention = attention_map[i]
            decoder_input = torch.cat((self.prenet(decoder_input), f0), dim=1)
            mel_output, gate_output, alignment = self.decode(decoder_input, attention)

            mel_outputs += [mel_output.squeeze(1)]
            gate_outputs += [gate_output]
            alignments += [alignment]

            decoder_input = mel_output

        mel_outputs, gate_outputs, alignments = self.parse_decoder_outputs(
            mel_outputs, gate_outputs, alignments
        )

        return mel_outputs, gate_outputs, alignments

# Cell
class Tacotron2(TTSModel):
    def __init__(self, hparams):
        super().__init__()
        self.mask_padding = hparams.mask_padding
        self.fp16_run = hparams.fp16_run
        self.include_f0 = hparams.include_f0
        self.n_mel_channels = hparams.n_mel_channels
        self.n_frames_per_step = hparams.n_frames_per_step
        self.embedding = nn.Embedding(hparams.n_symbols, hparams.symbols_embedding_dim)
        std = sqrt(2.0 / (hparams.n_symbols + hparams.symbols_embedding_dim))
        val = sqrt(3.0) * std  # uniform bounds for std
        self.embedding.weight.data.uniform_(-val, val)
        self.encoder = Encoder(hparams)
        self.decoder = Decoder(hparams)
        self.postnet = Postnet(hparams)
        if hparams.with_gst:
            self.gst = GST(hparams)
        self.speaker_embedding = nn.Embedding(
            hparams.n_speakers, hparams.speaker_embedding_dim
        )

    def parse_batch(self, batch):
        (
            text_padded,
            input_lengths,
            mel_padded,
            gate_padded,
            output_lengths,
            speaker_ids,
            *_,
        ) = batch
        if self.include_f0:
            f0_padded = batch[6]
            f0_padded = to_gpu(f0_padded).float()
        text_padded = to_gpu(text_padded).long()
        input_lengths = to_gpu(input_lengths).long()
        max_len = torch.max(input_lengths.data).item()
        mel_padded = to_gpu(mel_padded).float()
        gate_padded = to_gpu(gate_padded).float()
        output_lengths = to_gpu(output_lengths).long()
        speaker_ids = to_gpu(speaker_ids.data).long()
        ret_x = [
            text_padded,
            input_lengths,
            mel_padded,
            max_len,
            output_lengths,
            speaker_ids,
        ]
        if self.include_f0:
            ret_x.append(f0_padded)
        return (
            tuple(ret_x),
            (mel_padded, gate_padded),
        )

    def parse_output(self, outputs, output_lengths=None):
        if self.mask_padding and output_lengths is not None:
            mask = ~get_mask_from_lengths(output_lengths)
            mask = mask.expand(self.n_mel_channels, mask.size(0), mask.size(1))
            mask = mask.permute(1, 0, 2)

            outputs[0].data.masked_fill_(mask, 0.0)
            outputs[1].data.masked_fill_(mask, 0.0)
            outputs[2].data.masked_fill_(mask[:, 0, :], 1e3)  # gate energies

        return outputs

    def forward(self, inputs):
        (
            input_text,
            input_lengths,
            targets,
            max_len,
            output_lengths,
            speaker_ids,
            *_,
        ) = inputs
        if self.include_f0:
            f0s = inputs[6]
        else:
            f0s = None
        input_lengths, output_lengths = input_lengths.data, output_lengths.data

        embedded_inputs = self.embedding(input_text).transpose(1, 2)
        embedded_text = self.encoder(embedded_inputs, input_lengths)
        embedded_speakers = self.speaker_embedding(speaker_ids)[:, None]
        embedded_gst = self.gst(targets, output_lengths)
        embedded_gst = embedded_gst.repeat(1, embedded_text.size(1), 1)
        embedded_speakers = embedded_speakers.repeat(1, embedded_text.size(1), 1)

        encoder_outputs = torch.cat(
            (embedded_text, embedded_gst, embedded_speakers), dim=2
        )

        mel_outputs, gate_outputs, alignments = self.decoder(
            encoder_outputs, targets, memory_lengths=input_lengths, f0s=f0s
        )

        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments], output_lengths
        )

    def inference(self, inputs):
        text, style_input, speaker_ids, f0s = inputs
        embedded_inputs = self.embedding(text).transpose(1, 2)
        embedded_text = self.encoder.inference(embedded_inputs)
        embedded_speakers = self.speaker_embedding(speaker_ids)[:, None]
        if hasattr(self, "gst"):
            if isinstance(style_input, int):
                query = torch.zeros(1, 1, self.gst.encoder.ref_enc_gru_size).cuda()
                GST = torch.tanh(self.gst.stl.embed)
                key = GST[style_input].unsqueeze(0).expand(1, -1, -1)
                embedded_gst = self.gst.stl.attention(query, key)
            else:
                embedded_gst = self.gst(style_input)

        embedded_speakers = embedded_speakers.repeat(1, embedded_text.size(1), 1)
        if hasattr(self, "gst"):
            embedded_gst = embedded_gst.repeat(1, embedded_text.size(1), 1)
            encoder_outputs = torch.cat(
                (embedded_text, embedded_gst, embedded_speakers), dim=2
            )
        else:
            encoder_outputs = torch.cat((embedded_text, embedded_speakers), dim=2)

        mel_outputs, gate_outputs, alignments = self.decoder.inference(
            encoder_outputs, f0s
        )

        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments]
        )

    def inference_noattention(self, inputs):
        text, style_input, speaker_ids, f0s, attention_map = inputs
        embedded_inputs = self.embedding(text).transpose(1, 2)
        embedded_text = self.encoder.inference(embedded_inputs)
        embedded_speakers = self.speaker_embedding(speaker_ids)[:, None]
        if hasattr(self, "gst"):
            if isinstance(style_input, int):
                query = torch.zeros(1, 1, self.gst.encoder.ref_enc_gru_size).cuda()
                GST = torch.tanh(self.gst.stl.embed)
                key = GST[style_input].unsqueeze(0).expand(1, -1, -1)
                embedded_gst = self.gst.stl.attention(query, key)
            else:
                embedded_gst = self.gst(style_input)

        embedded_speakers = embedded_speakers.repeat(1, embedded_text.size(1), 1)
        if hasattr(self, "gst"):
            embedded_gst = embedded_gst.repeat(1, embedded_text.size(1), 1)
            encoder_outputs = torch.cat(
                (embedded_text, embedded_gst, embedded_speakers), dim=2
            )
        else:
            encoder_outputs = torch.cat((embedded_text, embedded_speakers), dim=2)

        mel_outputs, gate_outputs, alignments = self.decoder.inference_noattention(
            encoder_outputs, f0s, attention_map
        )

        mel_outputs_postnet = self.postnet(mel_outputs)
        mel_outputs_postnet = mel_outputs + mel_outputs_postnet

        return self.parse_output(
            [mel_outputs, mel_outputs_postnet, gate_outputs, alignments]
        )